In [71]:
!pip install -q geopy

In [1]:
import pandas as pd 
import time 
#geocode cities 
from geopy.geocoders import Nominatim

#to get combination
from itertools import product

#straight line distance conversion 
import math
from math import sin, cos, sqrt, atan2, radians
import numpy as np


In [2]:
capital_dic={
    'Alabama': 'Montgomery',
    'Alaska': 'Juneau',
    'Arizona':'Phoenix',
    'Arkansas':'Little Rock',
    'California': 'Sacramento',
    'Colorado':'Denver',
    'Connecticut':'Hartford',
    'Delaware':'Dover',
    'Florida': 'Tallahassee',
    'Georgia': 'Atlanta',
    'Hawaii': 'Honolulu',
    'Idaho': 'Boise',
    'Illinios': 'Springfield',
    'Indiana': 'Indianapolis',
    'Iowa': 'Des Monies',
    'Kansas': 'Topeka',
    'Kentucky': 'Frankfort',
    'Louisiana': 'Baton Rouge',
    'Maine': 'Augusta',
    'Maryland': 'Annapolis',
    'Massachusetts': 'Boston',
    'Michigan': 'Lansing',
    'Minnesota': 'St. Paul',
    'Mississippi': 'Jackson',
    'Missouri': 'Jefferson City',
    'Montana': 'Helena',
    'Nebraska': 'Lincoln',
    'Neveda': 'Carson City',
    'New Hampshire': 'Concord',
    'New Jersey': 'Trenton',
    'New Mexico': 'Santa Fe',
    'New York': 'Albany',
    'North Carolina': 'Raleigh',
    'North Dakota': 'Bismarck',
    'Ohio': 'Columbus',
    'Oklahoma': 'Oklahoma City',
    'Oregon': 'Salem',
    'Pennsylvania': 'Harrisburg',
    'Rhoda Island': 'Providence',
    'South Carolina': 'Columbia',
    'South Dakoda': 'Pierre',
    'Tennessee': 'Nashville',
    'Texas': 'Austin',
    'Utah': 'Salt Lake City',
    'Vermont': 'Montpelier',
    'Virginia': 'Richmond',
    'Washington': 'Olympia',
    'West Virginia': 'Charleston',
    'Wisconsin': 'Madison',
    'Wyoming': 'Cheyenne'  
} # create a dictionary, key is the state and value is the capital

capitals = pd.DataFrame.from_dict(capital_dic, orient='index', columns=["capital"])


In [3]:
capitals["state"] = capitals.index
capitals["forsearch"] = capitals[['capital', 'state']].apply(lambda x: ' '.join(x), axis=1)

In [4]:
capitals.head()

,capital,state,forsearch
Alabama,Montgomery,Alabama,Montgomery Alabama
Alaska,Juneau,Alaska,Juneau Alaska
Arizona,Phoenix,Arizona,Phoenix Arizona
Arkansas,Little Rock,Arkansas,Little Rock Arkansas
California,Sacramento,California,Sacramento California


In [5]:
def get_geocode(city):
    geolocator = Nominatim(user_agent='myapplication')
    location = geolocator.geocode(city)
    lat = location.raw["lat"]
    lon = location.raw["lon"]
    return lat+" "+lon

In [6]:
data1 = capitals[0:10]
data2 = capitals[10:20]
data3 = capitals[20:30]
data4 = capitals[30:40]
data5 = capitals[40:50]
data_arr = [data1["forsearch"], data2["forsearch"], data3["forsearch"], data4["forsearch"], data5["forsearch"]]


In [125]:
geo_arr = []

for sub in data_arr:   
    for x in sub:
        try:  
            geo_arr.append(get_geocode(x))
            time.sleep(1)
        except:
            geo_arr.append("na")
            time.sleep(1)

In [126]:
len(geo_arr)

50

In [9]:
capitals["geocode"] = geo_arr
capitals.to_csv("geocoded_capital.csv", index = False)


### In Excel, we process the missing geocodes and standardize geocodes as well 

In [8]:
capitals_geo = pd.read_csv("geocoded_capital.csv"); capitals_geo.head(2)

,capital,state,lat,lon
0,Montgomery,Alabama,32.366966,-86.300648
1,Juneau,Alaska,58.301950,-134.419734


### get distance between any two capitals

In [9]:
#to get combination of cities, join all text and ...
capitals_geo["lat"] = capitals_geo["lat"].astype(str)
capitals_geo["lon"] = capitals_geo["lon"].astype(str)

combination = list(capitals_geo[['capital', 'state', "lat", "lon"]].apply(lambda x: ','.join(x), axis=1))
# combination_df = pd.DataFrame({"text": combination})
combination[0:2]

['Montgomery,Alabama,32.3669656,-86.3006485',
 'Juneau,Alaska,58.30194960000001,-134.419734']

In [10]:
df = pd.DataFrame(data=list(product(combination, combination)), columns=['origin','destination'])
df.head(5)

,origin,destination
0,"Montgomery,Alabama,32.3669656,-86.3006485","Montgomery,Alabama,32.3669656,-86.3006485"
1,"Montgomery,Alabama,32.3669656,-86.3006485","Juneau,Alaska,58.30194960000001,-134.419734"
2,"Montgomery,Alabama,32.3669656,-86.3006485","Phoenix,Arizona,33.448586600000006,-112.0773456"
3,"Montgomery,Alabama,32.3669656,-86.3006485","Little Rock,Arkansas,34.7464809,-92.2895948"
4,"Montgomery,Alabama,32.3669656,-86.3006485","Sacramento,California,38.5815719,-121.49439960..."


In [11]:
df.to_csv("combination.csv", index=False)

### calculate straght Distance

In [17]:
#after seperating the string into columns in excel
distances = pd.read_csv("seperated.csv") 
print(distances.shape)
distances.head(2)

(2500, 8)


,origin_city,origin_state,origin_lat,origin_lon,destination_city,destination_state,destination_lat,destination_lon
0,Montgomery,Alabama,32.367,-86.3006,Montgomery,Alabama,32.3670,-86.3006
1,Montgomery,Alabama,32.367,-86.3006,Juneau,Alaska,58.3019,-134.4197


In [18]:
distances["lat1"]=np.radians(distances["origin_lat"])
# approximate radius of earth in km
R = 6373.0*0.621371

distances["lat1"] = np.radians(distances["origin_lat"])
distances["lon1"] = np.radians(distances["origin_lon"])
distances["lat2"] = np.radians(distances["destination_lat"])
distances["lon2"] = np.radians(distances["destination_lon"])

distances["dlon"] = distances["lon2"] - distances["lon1"]
distances["dlat"] = distances["lat2"] - distances["lat1"]

distances["a"] = np.sin(distances["dlat"] / 2)**2 + np.cos(distances["lat1"]) * np.cos(distances["lat2"]) * np.sin(distances["dlon"] / 2)**2
distances["c"] = 2 * np.arctan2(np.sqrt(distances["a"]), np.sqrt(1 - distances["a"]))
distances["distance_mile"] = R * distances["c"]


In [19]:
distances.drop(["lat1", "lon1", "lat2", "lon2", "dlon", "dlat", "a", "c"], axis=1, inplace=True)
distances = distances[distances["distance_mile"] > 0]
distances.head()

,origin_city,origin_state,origin_lat,origin_lon,destination_city,destination_state,destination_lat,destination_lon,distance_mile
1,Montgomery,Alabama,32.367,-86.3006,Juneau,Alaska,58.3019,-134.4197,2851.456965
2,Montgomery,Alabama,32.367,-86.3006,Phoenix,Arizona,33.4486,-112.0773,1493.745791
3,Montgomery,Alabama,32.367,-86.3006,Little Rock,Arkansas,34.7465,-92.2896,382.047267
4,Montgomery,Alabama,32.367,-86.3006,Sacramento,California,38.5816,-121.4944,2014.023878
5,Montgomery,Alabama,32.367,-86.3006,Denver,Colorado,39.7392,-104.9849,1158.601509


In [20]:
print(distances.shape)
distances_sorted = distances.sort_values(by="distance_mile");distances_sorted.head()

(2450, 9)


,origin_city,origin_state,origin_lat,origin_lon,destination_city,destination_state,destination_lat,destination_lon,distance_mile
1920,Providence,Rhoda Island,41.8249,-71.4136,Boston,Massachusetts,42.3603,-71.0583,41.247533
1038,Boston,Massachusetts,42.3603,-71.0583,Providence,Rhoda Island,41.8249,-71.4136,41.247533
957,Annapolis,Maryland,38.9786,-76.4928,Dover,Delaware,39.1582,-75.5244,53.426515
369,Dover,Delaware,39.1582,-75.5244,Annapolis,Maryland,38.9786,-76.4928,53.426515
1028,Boston,Massachusetts,42.3603,-71.0583,Concord,New Hampshire,43.2072,-71.5375,63.379658


In [21]:
end = 2450
values = []
for i in range(0,end):
    if i%2 == 0:
        values.append(0)
    else: 
        values.append(1)


In [22]:
distances_sorted["filter"] = values;distances_sorted.tail(2)

,origin_city,origin_state,origin_lat,origin_lon,destination_city,destination_state,destination_lat,destination_lon,distance_mile,filter
518,Honolulu,Hawaii,21.3251,-157.8466,Augusta,Maine,44.3106,-69.7797,5110.878966,0
910,Augusta,Maine,44.3106,-69.7797,Honolulu,Hawaii,21.3251,-157.8466,5110.878966,1


In [23]:
distances_sorted.to_csv("distance_sorted.csv")

In [72]:
distance_unique = distances_sorted[distances_sorted["filter"] == 1]
distance_unique.drop(["filter"], axis=1, inplace = True)
print(len(distance_unique))

1225


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [73]:
distance_unique.to_csv("distances.csv", index=False)
#removed duplicate distance in excel, A-B and B-A reduced to 1 distance

## Trips!!


In [36]:
distance_unique = pd.read_csv("distance_sorted.csv")

In [37]:
print(len(distance_unique))
distance_unique.head()

2450


,Unnamed: 0,origin_city,origin_state,origin_lat,origin_lon,destination_city,destination_state,destination_lat,destination_lon,distance_mile,filter
0,1920,Providence,Rhoda Island,41.8249,-71.4136,Boston,Massachusetts,42.3603,-71.0583,41.247533,0
1,1038,Boston,Massachusetts,42.3603,-71.0583,Providence,Rhoda Island,41.8249,-71.4136,41.247533,1
2,957,Annapolis,Maryland,38.9786,-76.4928,Dover,Delaware,39.1582,-75.5244,53.426515,0
3,369,Dover,Delaware,39.1582,-75.5244,Annapolis,Maryland,38.9786,-76.4928,53.426515,1
4,1028,Boston,Massachusetts,42.3603,-71.0583,Concord,New Hampshire,43.2072,-71.5375,63.379658,0


In [38]:
start_city = "Denver"
route = [start_city] #to acommandate firt city
total_distance = 0
# route[-1]

def optimize_distance(city):
    nth = 0
    global total_distance
    min_distance = distance_unique[distance_unique["origin_city"] == city]["distance_mile"].min()
    min_distance_city = distance_unique[distance_unique["distance_mile"] == min_distance]["destination_city"].values[0]
    while min_distance_city in route: 
        #take nth smallest distance to avoid duplicate route
        nth += 1 
        min_distance_row = distance_unique[distance_unique["origin_city"] == city].nsmallest(nth, 'distance_mile', keep='last').iloc[nth-1,]
        min_distance = min_distance_row["distance_mile"]
        min_distance_city = min_distance_row["destination_city"]
        
    route.append(min_distance_city)
    total_distance += min_distance    

for i in range(0,49): 
    optimize_distance(route[-1])
    
#add Sccramento as final destination
route.append(route[0])

#adding distance between Holonunu to Sacramento 
distance_end_origin = distance_unique[(distance_unique["origin_city"] == start_city) & (distance_unique["destination_city"] == route[-2])]["distance_mile"].values[0]
total_distance = total_distance + distance_end_origin

In [39]:
print(route)
print(total_distance)

['Denver', 'Cheyenne', 'Pierre', 'Bismarck', 'St. Paul', 'Madison', 'Springfield', 'Jefferson City', 'Topeka', 'Lincoln', 'Des Monies', 'Indianapolis', 'Frankfort', 'Columbus', 'Charleston', 'Richmond', 'Annapolis', 'Dover', 'Trenton', 'Harrisburg', 'Albany', 'Hartford', 'Providence', 'Boston', 'Concord', 'Montpelier', 'Augusta', 'Lansing', 'Nashville', 'Atlanta', 'Montgomery', 'Tallahassee', 'Columbia', 'Raleigh', 'Jackson', 'Baton Rouge', 'Little Rock', 'Oklahoma City', 'Austin', 'Santa Fe', 'Phoenix', 'Salt Lake City', 'Boise', 'Helena', 'Olympia', 'Salem', 'Carson City', 'Sacramento', 'Juneau', 'Honolulu', 'Denver']
19178.80260157722


In [40]:
print(len(route))
print(len(set(route)))


51
50


In [41]:
capital_cities = set(list(capitals["capital"]))
capital_cities - set(route)

set()

In [42]:
print(total_distance)
print(route)

19178.80260157722
['Denver', 'Cheyenne', 'Pierre', 'Bismarck', 'St. Paul', 'Madison', 'Springfield', 'Jefferson City', 'Topeka', 'Lincoln', 'Des Monies', 'Indianapolis', 'Frankfort', 'Columbus', 'Charleston', 'Richmond', 'Annapolis', 'Dover', 'Trenton', 'Harrisburg', 'Albany', 'Hartford', 'Providence', 'Boston', 'Concord', 'Montpelier', 'Augusta', 'Lansing', 'Nashville', 'Atlanta', 'Montgomery', 'Tallahassee', 'Columbia', 'Raleigh', 'Jackson', 'Baton Rouge', 'Little Rock', 'Oklahoma City', 'Austin', 'Santa Fe', 'Phoenix', 'Salt Lake City', 'Boise', 'Helena', 'Olympia', 'Salem', 'Carson City', 'Sacramento', 'Juneau', 'Honolulu', 'Denver']


## Testing - To automize and find shortest distance with all 50 states as origin city

In [32]:
capital_cities = list(capitals["capital"])
print(capital_cities)

['Montgomery', 'Juneau', 'Phoenix', 'Little Rock', 'Sacramento', 'Denver', 'Hartford', 'Dover', 'Tallahassee', 'Atlanta', 'Honolulu', 'Boise', 'Springfield', 'Indianapolis', 'Des Monies', 'Topeka', 'Frankfort', 'Baton Rouge', 'Augusta', 'Annapolis', 'Boston', 'Lansing', 'St. Paul', 'Jackson', 'Jefferson City', 'Helena', 'Lincoln', 'Carson City', 'Concord', 'Trenton', 'Santa Fe', 'Albany', 'Raleigh', 'Bismarck', 'Columbus', 'Oklahoma City', 'Salem', 'Harrisburg', 'Providence', 'Columbia', 'Pierre', 'Nashville', 'Austin', 'Salt Lake City', 'Montpelier', 'Richmond', 'Olympia', 'Charleston', 'Madison', 'Cheyenne']


In [59]:
result_distances = []
result_routes = []

for start_city in capital_cities: 
    route = [start_city] #to acommandate firt city
    total_distance = 0

    def optimize_distance(city):
        nth = 0
        global total_distance
        min_distance = distance_unique[distance_unique["origin_city"] == city]["distance_mile"].min()
        min_distance_city = distance_unique[distance_unique["distance_mile"] == min_distance]["destination_city"].values[0]
        while min_distance_city in route: 
            #take nth smallest distance to avoid duplicate route
            nth += 1 
            min_distance_row = distance_unique[distance_unique["origin_city"] == city].nsmallest(nth, 'distance_mile', keep='last').iloc[nth-1,]
            min_distance = min_distance_row["distance_mile"]
            min_distance_city = min_distance_row["destination_city"]
        
        route.append(min_distance_city)
        total_distance += min_distance    

    for i in range(0,49): 
        optimize_distance(route[-1])
    
    #add Sccramento as final destination
    route.append(route[0])

    #adding distance between Holonunu to Sacramento 
    distance_end_origin = distance_unique[(distance_unique["origin_city"] == start_city) & (distance_unique["destination_city"] == route[-2])]["distance_mile"].values[0]
    total_distance = total_distance + distance_end_origin
    
    result_distances.append(total_distance)
    result_routes.append(route)                       
                        

In [60]:
min_distance = min(result_distances)
optimum_route = result_routes[int(result_distances.index(min_distance))]
print(min_distance)
print(optimum_route)


18993.7967678579
['Phoenix', 'Santa Fe', 'Denver', 'Cheyenne', 'Pierre', 'Bismarck', 'St. Paul', 'Madison', 'Springfield', 'Jefferson City', 'Topeka', 'Lincoln', 'Des Monies', 'Indianapolis', 'Frankfort', 'Columbus', 'Charleston', 'Richmond', 'Annapolis', 'Dover', 'Trenton', 'Harrisburg', 'Albany', 'Hartford', 'Providence', 'Boston', 'Concord', 'Montpelier', 'Augusta', 'Lansing', 'Nashville', 'Atlanta', 'Montgomery', 'Tallahassee', 'Columbia', 'Raleigh', 'Jackson', 'Baton Rouge', 'Little Rock', 'Oklahoma City', 'Austin', 'Salt Lake City', 'Boise', 'Helena', 'Olympia', 'Salem', 'Carson City', 'Sacramento', 'Juneau', 'Honolulu', 'Phoenix']


In [71]:
df = pd.DataFrame({"seq": optimum_route})
merge_df = pd.merge(df, distance_unique, left_on="seq", right_on="origin_city", how="left"); merge_df.head()
merge_df = merge_df[["origin_city", "origin_state", "origin_lat", "origin_lon"]].drop_duplicates()
merge_df

,origin_city,origin_state,origin_lat,origin_lon
0,Phoenix,Arizona,33.4486,-112.0773
49,Santa Fe,New Mexico,35.6870,-105.9378
98,Denver,Colorado,39.7392,-104.9849
147,Cheyenne,Wyoming,41.1400,-104.8200
196,Pierre,South Dakoda,44.3684,-100.3540
245,Bismarck,North Dakota,46.8083,-100.7837
294,St. Paul,Minnesota,44.9504,-93.1015
343,Madison,Wisconsin,43.0748,-89.3838
392,Springfield,Illinios,39.7997,-89.6475
441,Jefferson City,Missouri,38.5774,-92.1724


In [72]:
merge_df.to_csv("optimum_route.csv", index=False)